Praca domowa 3 (8 pkt.)
Termin: 15.04.2021 15:00

Dla wybranego modelu przygotuj knitr/jupyter notebook w formacie html lub pdf rozwiązując poniższe punkty. Pracę domową należy zgłosić przez Pull Request na GitHub w folderze PraceDomowe/PracaDomowa3/Nazwisko_Imię.

    Dla wybranej obserwacji ze zbioru danych wylicz predykcję modelu.
    Dla wybranej obserwacji z punktu 1., wylicz dekompozycję predykcji modelu używając profili Ceteris Paribus/ ICE (pakiety w R: DALEX, ALEPlot, ingredients, pakiety w Python: dalex, pyCeterisParibus)
    Wybierz dwie obserwacje ze zbioru danych, które mają różne profile CP (na przykład: dla obserwacji A odpowiedź modelu rośnie wraz ze wzrostem wartości zmiennej, a dla obserwacji B maleje). Zauważ, żeby mieć takie różnice, musisz mieć model z interakcjami.
    Skomentuj poszczególne wyniki uzyskane w powyższych punktach.

W pracy domowej oceniane przede wszystkim będą wnioski z otrzymanych analiz. Kod potrzebny do reprudokowalności wyników również proszę załączyć.

Prace domowe należy zgłaszać przez Pull Request o tytule Nazwisko Imię PD3. Wszystkie pliki związane z pracą domową proszę o umieszczenie w folderze Nazwisko_Imię

In [ ]:
suppressWarnings(suppressMessages(library(OpenML)))
suppressWarnings(suppressMessages(library(mlr)))
suppressWarnings(suppressMessages(library(pROC)))
suppressWarnings(suppressMessages(library(plyr)))
suppressWarnings(suppressMessages(library(DALEX)))
suppressWarnings(suppressMessages(library(DALEXtra)))

set.seed(7)

In [ ]:
def <- read.csv("diabetes.csv")

def$class[def$class == "tested_positive"] <- 1
def$class[def$class == "tested_negative"] <- 0
head(def)

In [ ]:
m <- sample(1:nrow(def), 0.7*nrow(def))
train <- def[m,]
test <- def[-m,]

In [ ]:
classif_task <- makeClassifTask(data = train, target = "class")

classif_lrn <- makeLearner("classif.svm", predict.type = "prob")

In [ ]:
model <- train(classif_lrn, classif_task)

#     Dla wybranej obserwacji ze zbioru danych wylicz predykcję modelu.

In [ ]:
r <- test[sample(1:nrow(test), 1),]
r

In [ ]:
pred_r <- predict(model, newdata = r)$data$prob.1
pred_r

Model przewiduje dobrze tę obserwację

#     Dla wybranej obserwacji z punktu 1., wylicz dekompozycję predykcji modelu używając profili Ceteris Paribus/ ICE


In [ ]:
explainer <- explain(model = model,
                     data = def[,-9],
                     y = as.numeric(as.character(def$class)))

In [ ]:
parib <- predict_profile(explainer, r[, -9])
plot(parib)

Wartości zmiennych dla tej obserwacji i ich położenie na grafach wskazują na wysokie prawdopodobieństwo wystąpienia cukrzycy. W tym przypadku (lokalnym) liczba ciąż nie wpływa znacznie na predykcję modelu. Natomiast zmienne tj. age, insu, mass i plas w zależności od wartości mogą znacznie wpływać na predykcję. Niskie mass i plas zmniejszają prawdopodobieństwo gdy są nieduże, age i insu przeciwnie.

#    Wybierz dwie obserwacje ze zbioru danych, które mają różne profile CP

In [ ]:
paribA <- predict_profile(explainer, def[3, -9])
paribB <- predict_profile(explainer, def[5, -9])
plot(paribA,variables = "pedi")
plot(paribB,variables = "pedi")

In [ ]:
paribA <- predict_profile(explainer, def[616, -9])
paribB <- predict_profile(explainer, def[356, -9])
plot(paribA,variables = "preg")
plot(paribB,variables = "preg")

W pierwszym przykładzie wzrost zmiennej pedi a w drugim zmiennej preg wpływa odwrotnie na dane obserwacje. Dzieje się tak najprawdopodobniej z powodu różnych wartości pozostałych zmienneych w tych obserwacjach. Mogą one wchodzić w interakcję z interesującymi nas zmiennymi i przez to np. duża liczba ciąż różnie wpływa na predykcję w zależności od wartości innych zmiennych

In [ ]:
def[3, ]
def[5, ]

In [ ]:
def[616,]
def[356,]